In [7]:
!pip install turfpy --target C:\Users\marti\miniconda3\Lib\site-packages

  Using cached turfpy-0.0.7-py3-none-any.whl
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   -- ------------------------------------- 1.1/15.5 MB 69.1 MB/s eta 0:00:01
   ------ --------------------------------- 2.7/15.5 MB 33.7 MB/s eta 0:00:01
   ---------- ----------------------------- 4.1/15.5 MB 37.6 MB/s eta 0:00:01
   -------------- ------------------------- 5.6/15.5 MB 32.7 MB/s eta 0:00:01
   ------------------ --------------------- 7.0/15.5 MB 31.9 MB/s eta 0:00:01
   ---------------------- ----------------- 8.6/15.5 MB 34.2 MB/s eta 0:00:01
   -------------------------- ----------

In [2]:
import sysconfig; print(sysconfig.get_paths()["purelib"])

C:\Users\marti\miniconda3\Lib\site-packages


In [1]:
from selenium import webdriver
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import hashlib

In [28]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

In [4]:
driver.get('https://www.google.com/maps/search/restaurantes/@4.761191,-74.0557006,19z/data=!4m2!2m1!6e5?hl=es&entry=ttu')

In [39]:
def get_loc_google_url(url):
    url=url[url.find('!8m2!3d')+7:]
    return url[:url.find('!16s')].split('!4d')

In [78]:
def extract_data_restaurants(restaurants):
    
    scroll=driver.find_element(By.CLASS_NAME,'Nv2PK')
    iframe=driver.find_element(By.CLASS_NAME,'Nv2PK')

    scroll_origin = ScrollOrigin.from_element(iframe)

    for i in range(0,5):
        ActionChains(driver)\
            .move_to_element(scroll)\
            .scroll_from_origin(scroll_origin,0, 50000)\
            .perform()
    
    maps_results = driver.find_elements(By.CLASS_NAME,'Nv2PK')
    print(len(maps_results))
    for result in maps_results:
        restaurant = {}
        try:
            name = result.find_element(By.CLASS_NAME, 'qBF1Pd') 
            restaurant['name'] = name.text
        except NoSuchElementException:
            restaurant['name'] = None
            
        try:  
            rating = result.find_element(By.CLASS_NAME, 'MW4etd') 
            restaurant['rating'] = float(rating.text.replace(',','.'))
        except NoSuchElementException:
            restaurant['rating'] = None
            
        try:
            number_commnets_google = result.find_element(By.CLASS_NAME, 'UY7F9') 
            restaurant['number_commnets_google'] = int(number_commnets_google.text.replace("(", "").replace(")", "").replace(".", ""))
        except NoSuchElementException:
            restaurant['number_commnets_google'] = None
            
        try:
            address = result.find_elements(By.CLASS_NAME, 'W4Efsd') 
            line = address[2].text.split('·')
            
            if len(line)>1:
                restaurant['address'] = line[1]
                restaurant['type'] = line[0]
            elif len(line)== 1:
                restaurant['type'] = line[0]
                
        except NoSuchElementException:
            restaurant['address'] = None
            
        try:
            link= result.find_element(By.CLASS_NAME, 'hfpxzc')
            restaurant['google_maps_url'] = link.get_attribute('href')
            
            restaurant['latitud'] = get_loc_google_url(restaurant['google_maps_url'])[0]
            restaurant['longitud'] = get_loc_google_url(restaurant['google_maps_url'])[1]
        except NoSuchElementException:
            restaurant['google_maps_url'] = None
        
        restaurants.append(restaurant)
    return restaurants

In [79]:
def navigate_map(lat, lon, restaurants):
    driver.get('https://www.google.com/maps/search/restaurantes/@'+str(lat)+','+str(lon)+',19z/data=!4m2!2m1!6e5?hl=es&entry=ttu')
    restaurants=extract_data_restaurants(restaurants)
    return restaurants

In [80]:
lat = 4.6038511
lon = -74.0756006
restaurants = []

for i in range(0,5):
    lat = lat + i*0.001
    lon = -74.0756006
    for j in range(0,5):
        lon = lon + j*0.001
        print(str(lat),'-',str(lon))
        restaurants = navigate_map(lat, lon, restaurants) 
        
print("Scarping Ready")

4.6038511 - -74.0756006
20
4.6038511 - -74.0746006
20
4.6038511 - -74.0726006
20
4.6038511 - -74.0696006
20
4.6038511 - -74.0656006
20
4.6048511 - -74.0756006
20
4.6048511 - -74.0746006
20
4.6048511 - -74.0726006
20
4.6048511 - -74.0696006
20
4.6048511 - -74.0656006
20
4.6068511 - -74.0756006
20
4.6068511 - -74.0746006
20
4.6068511 - -74.0726006
20
4.6068511 - -74.0696006
20
4.6068511 - -74.0656006
20
4.6098511 - -74.0756006
20
4.6098511 - -74.0746006
20
4.6098511 - -74.0726006
20
4.6098511 - -74.0696006
20
4.6098511 - -74.0656006
20
4.6138511 - -74.0756006
20
4.6138511 - -74.0746006
20
4.6138511 - -74.0726006
20
4.6138511 - -74.0696006
20
4.6138511 - -74.0656006
20
Scarping Ready


In [82]:
df = pd.DataFrame(restaurants)
print(df.duplicated(keep='first').value_counts())
df=df.drop_duplicates().reset_index().drop(columns=['index'])
df

True     270
False    230
dtype: int64


,name,rating,number_commnets_google,address,type,google_maps_url,latitud,longitud
0,Restaurante Morena Parrilla,4.0,17.0,Cra. 12 # 15-25 Piso 2. Centro Comercial Gran...,Colombiana,https://www.google.com/maps/place/Restaurante+...,4.6039966,-74.0769667
1,RESTAURANTE GUSTA MAS,5.0,2.0,Cl. 16 #12-55,Colombiana,https://www.google.com/maps/place/RESTAURANTE+...,4.6050923,-74.0767801
2,Restaurante El Antojo de La 15,NaN,NaN,Cl. 15 #32,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603622,-74.076309
3,Restaurante Pescadero Del Centro,4.5,40.0,Cra. 9 #58,Marisco,https://www.google.com/maps/place/Restaurante+...,4.6039659,-74.074317
4,Restaurante Cafetería Aqúm,NaN,NaN,NaN,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.6037598,-74.0759631
...,...,...,...,...,...,...,...,...
225,Anna & Otto Macarena,4.6,672.0,Cra. 4a #26c-35,Pizza,https://www.google.com/maps/place/Anna+%26+Ott...,4.613256,-74.0665989
226,Pastis Ristorante,4.4,46.0,Cra. 4a #27 - 86,Italiana,https://www.google.com/maps/place/Pastis+Risto...,4.614742,-74.0660848
227,Restaurante rincón Casa de Paella,4.5,172.0,Cl. 26c #4a-15,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.6127433,-74.0667679
228,Bangkok Thai Cuisine,4.5,192.0,Cl. 26c #4-41,Tailandesa,https://www.google.com/maps/place/Bangkok+Thai...,4.6126938,-74.0665223


In [83]:
def create_hash_id(value):
    return hashlib.sha256(str.encode(str(value))).hexdigest()[0:32]

In [91]:
df['hash_id'] = df['name']+ df['address']
df['hash_id'] = df['hash_id'].apply(create_hash_id)
df

,name,rating,number_commnets_google,address,type,google_maps_url,latitud,longitud,hash_id
0,Restaurante Morena Parrilla,4.0,17.0,Cra. 12 # 15-25 Piso 2. Centro Comercial Gran...,Colombiana,https://www.google.com/maps/place/Restaurante+...,4.6039966,-74.0769667,027bcb6af9d1be568cf1ae518fa7c10f
1,RESTAURANTE GUSTA MAS,5.0,2.0,Cl. 16 #12-55,Colombiana,https://www.google.com/maps/place/RESTAURANTE+...,4.6050923,-74.0767801,c52a652d8a563dc0cd251ddf5814bbe2
2,Restaurante El Antojo de La 15,NaN,NaN,Cl. 15 #32,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603622,-74.076309,b6bc338fc1a8d79b22083f3d315dc6be
3,Restaurante Pescadero Del Centro,4.5,40.0,Cra. 9 #58,Marisco,https://www.google.com/maps/place/Restaurante+...,4.6039659,-74.074317,93cb8caef719bf99a37bb12b503ee75a
4,Restaurante Cafetería Aqúm,NaN,NaN,NaN,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.6037598,-74.0759631,9b2d5b4678781e53038e91ea5324530a
...,...,...,...,...,...,...,...,...,...
225,Anna & Otto Macarena,4.6,672.0,Cra. 4a #26c-35,Pizza,https://www.google.com/maps/place/Anna+%26+Ott...,4.613256,-74.0665989,f015860f80a926525974517549239c6b
226,Pastis Ristorante,4.4,46.0,Cra. 4a #27 - 86,Italiana,https://www.google.com/maps/place/Pastis+Risto...,4.614742,-74.0660848,41462500978ad1b6864f594a603b3061
227,Restaurante rincón Casa de Paella,4.5,172.0,Cl. 26c #4a-15,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.6127433,-74.0667679,e01677ef20fc43fcf6dea862c590e0b9
228,Bangkok Thai Cuisine,4.5,192.0,Cl. 26c #4-41,Tailandesa,https://www.google.com/maps/place/Bangkok+Thai...,4.6126938,-74.0665223,b45cbcc226145cbdceae9bb2a8a87c33


In [86]:
def extract_and_combine(row):
    cell_values = [row['latitud'], row['longitud']]
    return cell_values

In [89]:
aux = pd.DataFrame()
aux['center'] = df.apply(extract_and_combine, axis=1)
aux

,center
0,"[4.6039966, -74.0769667]"
1,"[4.6050923, -74.0767801]"
2,"[4.603622, -74.076309]"
3,"[4.6039659, -74.074317]"
4,"[4.6037598, -74.0759631]"
...,...
225,"[4.613256, -74.0665989]"
226,"[4.614742, -74.0660848]"
227,"[4.6127433, -74.0667679]"
228,"[4.6126938, -74.0665223]"


In [ ]:
def find_polygon_containing_point(geojson, coordinates):
    point_feature = point([coordinates['lng'], coordinates['lat']])
    
    for feature in geojson:
        try:
            polygon_feature = polygon([feature['geometry']['coordinates']])
            if boolean_point_in_polygon(point_feature, polygon_feature):
                return feature
        except Exception as e:
            print("Upz not available")
    
    return None  # No polygon found

In [96]:
df.to_json(path_or_buf='D:/MarketResearch/Restaurantes/restaurants_sample.json', orient='records')

In [521]:
df['type'].value_counts()

Restaurante              123
Colombiana                40
Restaurante               21
Italiana                   7
Vegetariana                4
Mexicana                   4
Marisco                    3
Comida rápida              3
Desayuno                   3
China                      3
Peruana                    3
Pizza                      2
Bar de tapas               1
Parrilla                   1
Española                   1
Fusión asiática            1
Caribeña                   1
Serbia                     1
India                      1
Ideal para familias        1
Vegetariana estricta       1
Libanesa                   1
Criolla                    1
Tailandesa                 1
Name: type, dtype: int64